# このファイルについて
- author: 松永

In [1]:
import numpy as np
import pandas as pd
import cudf
import missingno as msno
import matplotlib.pyplot as plt

In [2]:
import cupy
cupy.cuda.Device(1).use()

<CUDA Device 1>

In [3]:
# 道路名
TARGET_ROAD = 'kannetsu'
# TARGET_ROAD = 'touhoku'

PERIOD = '20230401-20230930'

# 交通量
TRAFFIC_DIR = f'./traffic'
TRAFFIC_CSV = f'{TRAFFIC_DIR}/{TARGET_ROAD}_{PERIOD}_2KP.csv'

# 欠損値を確認

### 長期間にわたって線形補間できない欠損が存在することを確認

In [7]:
ROADS = ['kannetsu', 'touhoku']
PERIODS = [
    '20210401-20220331',
    '20220401-20230331',
    '20230401-20230930',
]

In [7]:
def col_interpolate(df: cudf.DataFrame, cols: list, method='linear'):
    """
    dfのcol列内の欠損を区間ごとに線形補間する
    """
    grouped_intp = lambda g: g.reset_index(drop=True).loc[:, cols].interpolate(method=method, axis=0)
    
    df = df.sort_values(['start_code', 'end_code', 'KP', 'datetime']).reset_index(drop=True)
    interpolated = (df.to_pandas()
                    .groupby(['start_code', 'end_code', 'KP'])
                    .apply(grouped_intp)
                    .sort_index()
                    .reset_index())
    
    df_res = df.assign(**{col: interpolated[col] for col in cols})
    return df_res

In [9]:
for road in ROADS:
    for period in PERIODS:
        print('='*30, road, period, '='*30)

        df_traffic = cudf.from_pandas(
            pd.read_csv(
                f'{TRAFFIC_DIR}/{road}_{period}_2KP.csv', parse_dates=True, index_col='datetime', 
                dtype={'start_code': str, 'end_code': str,}
            )
        ).reset_index()

        df_traffic2 = col_interpolate(df_traffic, cols=['speed', 'OCC', 'allCars'], method='linear')

        # 線形補間ができなかった箇所を列挙
        for g_name, df_g in df_traffic2.loc[df_traffic2.speed.isna()].to_pandas().groupby(['start_name', 'end_name', 'KP']):
            print('-'*20, g_name, '-'*20)
            print('# of null:', df_g.shape[0])
            print('Period:', f'{df_g.datetime.iloc[0]} --> {df_g.datetime.iloc[-1]}')
            print()

        del df_traffic, df_traffic2

============================== 20210401-20220331 ==============================
-------------------- ('久喜', '加須', 27.203) --------------------
# of null: 102936
Period: 2021-04-01 00:00:00 --> 2022-03-24 09:55:00

-------------------- ('久喜白岡ＪＣＴ', '久喜', 24.999) --------------------
# of null: 105120
Period: 2021-04-01 00:00:00 --> 2022-03-31 23:55:00

-------------------- ('佐野藤岡', '佐野ＳＡ', 57.796) --------------------
# of null: 20004
Period: 2021-04-01 00:00:00 --> 2021-06-09 10:55:00

-------------------- ('佐野藤岡', '館林', 54.4) --------------------
# of null: 10524
Period: 2021-04-01 00:00:00 --> 2021-05-07 12:55:00

-------------------- ('加須', '久喜', 27.203) --------------------
# of null: 77064
Period: 2021-04-01 00:00:00 --> 2021-12-24 13:55:00

-------------------- ('加須', '羽生', 36.255) --------------------
# of null: 10500
Period: 2021-04-01 00:00:00 --> 2021-05-07 10:55:00

-------------------- ('宇都宮', '鹿沼', 100.261) --------------------
# of null: 105120
Period: 2021-04-01 00:00:00 

### 速度の欠損パターンを確認する

In [8]:
road = 'kannetsu'
# road = 'touhoku'

period = '20230401-20230930'

In [9]:
df_traffic = pd.read_csv(f'{TRAFFIC_DIR}/{road}_{period}_2KP.csv', dtype={'start_code': str, 'end_code': str,})
df_traffic.head(3)

,datetime,start_name,end_name,start_code,end_code,start_pref_code,end_pref_code,start_lat,end_lat,start_lng,...,lane_count,KP_distance,KP,start_KP,end_KP,direction,limit_speed,OCC,allCars,speed
0,2023-04-01 00:00:00,所沢,大泉ＪＣＴ,1800006,1110210,11,13,35.80615,35.75582,139.535511,...,3,8.6,2.26,9.4,0.8,up,100.0,3.0,91.0,87.0
1,2023-04-01 00:00:00,大泉ＪＣＴ,所沢,1110210,1800006,13,11,35.75582,35.80615,139.601514,...,3,8.6,2.48,0.8,9.4,down,100.0,3.0,78.0,96.0
2,2023-04-01 00:00:00,大泉ＪＣＴ,所沢,1110210,1800006,13,11,35.75582,35.80615,139.601514,...,3,8.6,3.90,0.8,9.4,down,100.0,3.0,87.0,84.0


In [9]:
def create_na_table(df: pd.DataFrame):
    na_table = df.isna().sum()
    na_percent_table = (df.isna().sum() / len(df_traffic)) * 100
    
    na_table = (pd.concat([na_table, na_percent_table], axis=1)
                .rename(columns={0: '# of null', 1: '% of null'})
                .pipe(lambda _df: _df.loc[_df['# of null'] > 0])
                .sort_values('% of null'))
    return na_table

In [10]:
create_na_table(df_traffic)

,# of null,% of null
OCC,295800,3.792214
allCars,295800,3.792214
speed,304229,3.900276


In [11]:
cond1 = df_traffic.speed.isna() & df_traffic.OCC.isna()
cond2 = df_traffic.speed.isna() & df_traffic.OCC.notna()
cond3 = df_traffic.speed.isna() & df_traffic.allCars.notna()

df_tmp1 = df_traffic.loc[cond1]
print(len(df_tmp1))
df_tmp2 = df_traffic.loc[cond2]
print(len(df_tmp2))
df_tmp3 = df_traffic.loc[cond3]
print(len(df_tmp3))

295800
8429
8429


In [15]:
df_tmp2.loc[:, ['OCC', 'allCars']].value_counts().reset_index().rename(columns={0: '# of null (speed)'})

,OCC,allCars,# of null (speed)
0,0.0,0.0,8399
1,0.0,1.0,20
2,1.0,1.0,6
3,0.0,2.0,2
4,2.0,3.0,1
5,20.0,1.0,1


In [13]:
df_tmp3.loc[:, ['OCC', 'allCars']].value_counts().reset_index()

,OCC,allCars,0
0,0.0,0.0,8399
1,0.0,1.0,20
2,1.0,1.0,6
3,0.0,2.0,2
4,2.0,3.0,1
5,20.0,1.0,1


In [70]:
df_tmp3.loc[:, ['OCC', 'allCars']].value_counts().reset_index()

,OCC,allCars,0
0,0.0,0.0,5597
1,1.0,1.0,9
2,2.0,1.0,8
3,0.0,1.0,4
4,3.0,2.0,2
5,4.0,2.0,2
6,5.0,1.0,2
7,4.0,1.0,1
8,7.0,1.0,1
9,9.0,2.0,1


### 道路別の欠損割合を確認

In [5]:
road = 'touhoku'

periods = [
    '20210401-20220331',
    '20220401-20230331',
    '20230401-20230930'
]

In [7]:
df_traffic = pd.DataFrame()

for period in periods:
    print(f'Loading {period}...')
    _df_traffic = pd.read_csv(f'{TRAFFIC_DIR}/{road}_{period}_2KP.csv', dtype={'start_code': str, 'end_code': str,})
    print(f'Finished.')
    df_traffic = pd.concat([df_traffic, _df_traffic])

Loading 20210401-20220331...
Finished.
Loading 20220401-20230331...
Finished.
Loading 20230401-20230930...
Finished.


In [8]:
def create_na_table(df: pd.DataFrame):
    na_table = df.isna().sum()
    na_percent_table = (df.isna().sum() / len(df_traffic)) * 100
    
    na_table = (pd.concat([na_table, na_percent_table], axis=1)
                .rename(columns={0: '# of null', 1: '% of null'})
                .pipe(lambda _df: _df.loc[_df['# of null'] > 0])
                .sort_values('% of null'))
    return na_table

5分間での欠損割合

In [9]:
df_traffic.shape

(38915712, 29)

In [10]:
create_na_table(df_traffic)

,# of null,% of null
OCC,1594636,4.097666
allCars,1594636,4.097666
speed,1612447,4.143434


1時間での欠損割合

In [11]:
AGG_COLS = ['allCars', 'speed']
AGG_COLS_METHODS = {
    'allCars': 'sum', 
    'accum_speed': 'sum', 
}

In [12]:
sampling_rate = '1h'

In [13]:
# 1時間単位でそのままデータを取得すればいいカラム（静的カラム + 1日単位の時間指定なし検索数）
df_static = cudf.from_pandas(
    df_traffic
    .drop(AGG_COLS, axis=1)
    .assign(datetime=pd.to_datetime(df_traffic.datetime))
    .set_index('datetime')
    .groupby(['start_code', 'end_code', 'KP'])
    .apply(lambda g: g.asfreq(sampling_rate))
    .reset_index('datetime')
    .reset_index(drop=True)
)

df_static.head(3)

,datetime,start_name,end_name,start_code,end_code,start_pref_code,end_pref_code,start_lat,end_lat,start_lng,...,hour,minute,lane_count,KP_distance,KP,start_KP,end_KP,direction,limit_speed,OCC
0,2021-04-01 00:00:00,浦和本線,岩槻,1040013,1040016,11,11,35.89012,35.9347,139.723697,...,0,0,3,5.7,5.47,4.8,10.5,down,100.0,2.0
1,2021-04-01 01:00:00,浦和本線,岩槻,1040013,1040016,11,11,35.89012,35.9347,139.723697,...,1,0,3,5.7,5.47,4.8,10.5,down,100.0,2.0
2,2021-04-01 02:00:00,浦和本線,岩槻,1040013,1040016,11,11,35.89012,35.9347,139.723697,...,2,0,3,5.7,5.47,4.8,10.5,down,100.0,2.0


In [14]:
df_agg = cudf.from_pandas(
    df_traffic
    .assign(accum_speed=df_traffic['allCars'] * df_traffic['speed'])
    .assign(datetime=pd.to_datetime(df_traffic.datetime))
    .set_index('datetime')
    .groupby(['start_code', 'end_code', 'KP'])
    .apply(lambda g: g.resample(sampling_rate).agg(AGG_COLS_METHODS))
    .reset_index()
)

df_agg.head(3)

,start_code,end_code,KP,datetime,allCars,accum_speed
0,1040013,1040016,5.47,2021-04-01 00:00:00,558.0,48791.0
1,1040013,1040016,5.47,2021-04-01 01:00:00,502.0,43085.0
2,1040013,1040016,5.47,2021-04-01 02:00:00,447.0,37685.0


In [15]:
df = df_static.merge(df_agg, on=['datetime', 'start_code', 'end_code', 'KP'])
df = df.assign(speed=df.accum_speed / (df.allCars + 1)).drop(['accum_speed'], axis=1)

# カラムの順番を揃える
df = df.loc[:, df_traffic.columns]
df.head()

,datetime,start_name,end_name,start_code,end_code,start_pref_code,end_pref_code,start_lat,end_lat,start_lng,...,lane_count,KP_distance,KP,start_KP,end_KP,direction,limit_speed,OCC,allCars,speed
0,2021-12-15 00:00:00,浦和本線,岩槻,1040013,1040016,11,11,35.89012,35.9347,139.723697,...,3,5.7,5.47,4.8,10.5,down,100.0,2.0,558.0,85.912343
1,2021-12-15 01:00:00,浦和本線,岩槻,1040013,1040016,11,11,35.89012,35.9347,139.723697,...,3,5.7,5.47,4.8,10.5,down,100.0,2.0,484.0,84.395876
2,2021-12-15 02:00:00,浦和本線,岩槻,1040013,1040016,11,11,35.89012,35.9347,139.723697,...,3,5.7,5.47,4.8,10.5,down,100.0,1.0,465.0,83.912017
3,2021-12-15 03:00:00,浦和本線,岩槻,1040013,1040016,11,11,35.89012,35.9347,139.723697,...,3,5.7,5.47,4.8,10.5,down,100.0,2.0,601.0,82.744186
4,2021-12-15 04:00:00,浦和本線,岩槻,1040013,1040016,11,11,35.89012,35.9347,139.723697,...,3,5.7,5.47,4.8,10.5,down,100.0,3.0,813.0,83.422604


In [16]:
# NaNはリサンプリングすると0になる
na_condition = df.speed == 0

# 制限速度で置換
# df.loc[na_condition, 'speed'] = df.loc[na_condition, 'limit_speed']

In [17]:
n_na = len(df.loc[na_condition])
percent_na = n_na / len(df) * 100

print(f'全体レコード数: {len(df)} | 欠損レコード数: {n_na} | 欠損レコード割合: {percent_na:.3f}[%]')

全体レコード数: 3242976 | 欠損レコード数: 133282 | 欠損レコード割合: 4.110[%]


### 速度の欠損のみを制限速度で埋める
→ resampling後に行う

In [68]:
# def create_na_table(df: cudf.DataFrame):
#     na_table = df.isna().sum()
#     na_percent_table = (df.isna().sum() / len(df_traffic)) * 100
    
#     na_table = (cudf.concat([na_table, na_percent_table], axis=1)
#                 .rename(columns={0: '# of null', 1: '% of null'})
#                 .pipe(lambda _df: _df.loc[_df['# of null'] > 0])
#                 .sort_values('% of null'))
#     return na_table

In [69]:
# create_na_table(df_traffic)

In [70]:
# na_condition = df_traffic.speed.isna()

# # 制限速度で置換
# df_traffic.loc[na_condition, 'speed'] = df_traffic.loc[na_condition, 'limit_speed']

In [71]:
# create_na_table(df_traffic)

In [72]:
# %time msno.matrix(df_traffic.to_pandas(), labels=True)
# plt.show()